In [1]:
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchsummary import summary
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Clone yolov5 and install dependencies
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -qr requirements.txt
%cd ..

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
/content


In [4]:
import sys
sys.path.insert(0, "/content/yolov5")

In [5]:
# Set device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [6]:
# data_dir = "../data/spectrograms_split"
data_dir = "/content/drive/MyDrive/spectrograms_split"
train_dir = os.path.join(data_dir, "train")
val_dir   = os.path.join(data_dir, "val")
test_dir  = os.path.join(data_dir, "test")

In [7]:
# Define transformations (resize → to tensor → normalize with ImageNet stats)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # YOLOv5‐cls (like ResNet) expects 224×224 inputs
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# The transforms.ToTensor() converts a PIL image (0–255 ints) → FloatTensor (0.0–1.0)
# and rearranges (H, W, C) → (C, H, W). Normalization uses ImageNet mean/std to match
# pretrained YOLOv5s‐cls backbone expectations.

In [8]:
# Load train/validation/test datasets via ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset   = datasets.ImageFolder(val_dir,   transform=transform)
test_dataset  = datasets.ImageFolder(test_dir,  transform=transform)

# This automatically assigns labels based on subfolder names (normal, mild, moderate, severe).
# Example: train_dataset.classes == ['moderate', 'mild', 'normal', 'severe']

In [9]:
# Create DataLoaders (batch_size=32, shuffle during training)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,  num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, num_workers=2)

In [10]:
# Import & instantiate YOLOv5s‐CLS model
# (214 layers, ~7 M parameters). We load the pretrained weights from yolov5/weights.
from models.yolo import Model